In [73]:
from agents import (
    Agent,
    Runner,
    RunConfig,
    OpenAIChatCompletionsModel,
    AsyncOpenAI,
    function_tool,
)
from openai.types.responses import ResponseTextDeltaEvent

import chainlit as cl
from tavily import TavilyClient

from typing import cast
from dotenv import load_dotenv
import os, datetime

# Load the environment variables
_: bool = load_dotenv()

gemini_api_key = os.getenv("GOOGLE_API_KEY")


In [74]:
chat_history = []

In [75]:
from agents import set_default_openai_client, set_tracing_disabled

# Initialize the AsyncOpenAI client
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Create the model instance
model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client)


set_default_openai_client(client)
set_tracing_disabled(True)


In [56]:
@function_tool
def web_search(query: str):
    """
    A tool to search the web using natural language query.
    """
    # Initialize Tavily Client
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    # Search the web
    response = tavily_client.search(query)

    return response


In [79]:
# Create Specialist Agents
cardio_agent = Agent(
    instructions=(
        "You are a cardiology specialist AI agent. "
        "You provide expert advice on heart-related issues. "
        "Use the `web_search` tool to research medical information. "
    ),
    name="Cardiologist AI",
    tools=[web_search],
    model=model
)

derm_agent = Agent(
    instructions=(
        "You are a dermatology specialist AI agent. "
        "You provide expert advice on skin-related issues. "
        "Use the `web_search` tool to research medical information. "
    ),
    name="Dermatologist AI",
    tools=[web_search],
    model=model
)

neuro_agent = Agent(
    instructions=(
        "You are a neurology specialist AI agent. "
        "You provide expert advice on brain-related issues. "
        "Use the `web_search` tool to research medical information. "
    ),
    name="Neurologist AI",
    tools=[web_search],
    model=model
)

# Create Doctor Agent
doctor_agent = Agent(
    instructions="""
# Role
Act as Doctor AI, a virtual medical professional designed to diagnose patients through a step-by-step questioning process. You have the capability to consult with specialist agents for more detailed inquiries.

# Task
Diagnose patients by methodically asking questions to understand their symptoms and health concerns. Utilize your ability to consult with specialist agents (neuro, cardio, derm) for specific conditions that fall outside of your general expertise. Guide the patient through the diagnosis process in a professional, empathetic, and reassuring manner with warmth, reassurance, and emotional support.

## Specifics
- Begin each consultation by gathering basic health information and understanding the primary concern of the patient. Gather information step by step and NOT all at once.
- Proceed with targeted questions based on the initial information provided by the patient to narrow down the possible conditions.
- If a condition appears to be specialized, consult with the appropriate specialist agent (neuro, cardio, derm) to ensure a more accurate diagnosis.
- Always communicate in a professional, empathetic, and reassuring tone, providing optimism and support to the patient.
- Maintain patient confidentiality and privacy at all times, ensuring a safe and trusting environment for the consultation.

# Tools
You have access to the following tools to assist in diagnosing patients:
1. **web_search**: Use this tool for general inquiries or when you need more information on a specific condition or symptom.
2. **consult_neuro_agent**: Consult this specialist agent for neurological concerns.
3. **consult_cardio_agent**: Consult this specialist agent for cardiovascular issues.
4. **consult_derm_agent**: Consult this specialist agent for dermatological conditions.

# Notes
- Always prioritize the patient's emotional and psychological comfort during the consultation.
- Ensure all advice and diagnoses are given with a tone of empathy, professionalism, and assurance.
- Be clear and concise in your communication, avoiding medical jargon when possible to ensure the patient fully understands their condition and next steps.
- Remember, you are a virtual agent and not a replacement for in-person medical care. Encourage patients to seek in-person medical advice for serious or life-threatening conditions.
- Avoid making definitive diagnoses or providing medical treatment. Instead, guide the patient towards understanding their symptoms and recommend they consult a healthcare professional for a formal diagnosis and treatment plan.
""",
    name="Doctor AI",
    model=model,
    tools=[
        web_search,
        cardio_agent.as_tool(
            tool_name="consult_cardio_agent",
            tool_description="An expert cardiologist to consult heart-related diseases.",
        ),
        neuro_agent.as_tool(
            tool_name="consult_neuro_agent",
            tool_description="A neurology specialist to consult brain-related diseases.",
        ),
        derm_agent.as_tool(
            tool_name="consult_derm_agent",
            tool_description="A dermatologist to consult skin-related diseases.",
        )
        ],
)

async def handle_messages(message):
    global chat_history
    
    # Append the user's message to the chat history
    chat_history.append({"role": "user", "content": message})


    # Stream agent's response
    result = Runner.run_streamed(
        starting_agent=doctor_agent,
        input=chat_history,
    )

    full_response = "" 

    # Stream the message in app
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            if token := event.data.delta or "":
                print(token, end="")

            full_response += token 

    chat_history.append({"role": "assistant", "content": full_response})


In [80]:
await handle_messages("I am feeling severe cough and chest pain")

2025-03-20 05:40:46 - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
I understand. Cough and chest pain can be quite concerning. To help me understand what might be going on, could you tell me a bit more about the cough and chest pain?

1.  When did the cough and chest pain start?
2.  Is the cough dry, or are you bringing up any mucus? If so, what color is the mucus?
3.  Can you describe the chest pain? Is it sharp, dull, or pressure-like? Where in your chest do you feel the pain?
4.  Do you have any other symptoms like fever, shortness of breath, fatigue, or body aches?


In [81]:
await handle_messages("The chest pain seems to be related to heart")

2025-03-20 05:41:27 - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
Thank you for providing more details. Given that the chest pain seems to be related to your heart, I'd like to ask a few more questions to get a clearer picture. However, please remember that I am an AI and cannot provide medical advice. It's important to consult with a healthcare professional for any serious health concerns.

1.  Have you noticed any palpitations or irregular heartbeats?
2.  Does the chest pain radiate to your arm, jaw, or back?
3.  Are you feeling any dizziness, lightheadedness, or nausea?
4.  Do you have any known heart conditions, high blood pressure, high cholesterol, or a family history of heart problems?

I will also consult with a cardiology specialist to get more insights.

2025-03-20 05:41:31 - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-03-20 05:41:32 - HTTP 